In [67]:
%load_ext autoreload
%autoreload 2

import os
import cv2
import matplotlib.pyplot as plt
from attention.img_proc.img_process import *
from attention.img_proc.img_split import *
from attention.models.face_models import *
from attention.utils.img_plot import *
from attention.utils.utilities import *
from attention.params import *
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# Get attention_data directory
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, os.pardir, "attention_data")

In [71]:
# Converting video to frames:
video_folder_path = os.path.join(data_directory, "video")
video_name = "Midbatch.MOV"
video_path = os.path.join(video_folder_path, video_name)
frames = extract_video_frames(video_path, 
                              period_sec=1,
                              start_sec=60, end_sec=75)

# Saving the frames
image_folder_path = os.path.join(data_directory, "frames")
for idx, frame in enumerate(frames):
    save_image(frame, video_name.split('.')[0] + f'_f{idx}.png', image_folder_path)

In [64]:
# Training the face recognition model
known_folder_path = os.path.join(data_directory, "known_faces")
known_paths = load_image_paths(known_folder_path)
known_names = list(known_paths.keys())
known_faces = [read_image(image_path) for image_path in known_paths.values()]
known_encodings = train_faces(known_faces, known_names)

In [65]:
# PARAMS:
n_split_w = 6
n_split_h = 6
landmark_idx = LEFT_EYE_EDGES  + LEFT_IRIS_CENTER + RIGHT_EYE_EDGES + RIGHT_IRIS_CENTER

In [66]:
image_folder_path = os.path.join(data_directory, "frames")
image_paths = load_image_paths(image_folder_path)

for image_name, image_path in image_paths.items():
    # Loading image
    image = read_image(image_path)
    # Splitting image
    crops = split_image(image, n_split_w, n_split_h, 0.05, 0.05)
    # Generating bboxes for each crop
    bbox_crop_list = []
    bbox_crop_list_absolute = []
    for crop in crops:
        coord_set = detect_face(crop.image)
        bbox_crop_list.append(coord_set)
        coord_set_absolute = reconstruct_coord(crop, coord_set)
        bbox_crop_list_absolute.append(coord_set_absolute)
    bbox_crop_list_absolute
    # Eliminating duplicates bboxes
    bbox_list = bbox_merge(bbox_crop_list_absolute, intersect_threshold=0.6)
    # Drawing the unique bboxes on the original image
    image_output = annotate_bboxes(image, bbox_list)
    # Saving the image with its bboxes
    bbox_path = os.path.join(data_directory, "output_bbox")
    save_image(image_output, image_name + '.png', bbox_path)
    
    # Generating face crops
    faces = crop_faces(image, bbox_list)
    # Saving face crops
    face_path = os.path.join(data_directory, "face_crops")
    for idx, face in enumerate(faces):
        face_name = image_name + f'_{idx}'
        save_image(face, face_name + '.png', face_path)
    
    # Generating eye and iris landmarks
    for idx, face in enumerate(faces):
        face_name = image_name + f'_{idx}'
        mp_landmarks = find_landmarks(face)
        if mp_landmarks:                # Only run attention/ recognition if it detects a face
            # Converting the Mediapipe landmark to a standard system of coordinates
            landmark_list = convert_landmarks(face, mp_landmarks)
            # Drawing the face mesh on the face
            face_mesh = annotate_mesh(face, mp_landmarks)
            # Saving face with complete mesh
            mesh_path = os.path.join(data_directory, "output_mesh")
            save_image(face_mesh, face_name + '.png', mesh_path)
            
            # Detecting eye direction and attention
            face_name = image_name + f'_{idx}'
            eye_directions = detect_eye_directions(landmark_list, threshold = 0.63)
            attention = is_attentive(eye_directions)
            # Drawing iris landmarks + annotating attention results on original image
            prediction_left, score_left = eye_directions['left']
            prediction_right, score_right = eye_directions['right']
            prediction_attention = 'attentive' if attention else 'inattentive'
            face_attention = annotate_iris_attention(face, landmark_list, 
                                                    prediction_left, score_left, 
                                                    prediction_right, score_right,
                                                    prediction_attention)
            # Saving attention image output
            attention_path = os.path.join(data_directory, "output_attention")
            save_image(face_attention, face_name + '.png', attention_path)
        
            # Recognizing a face
            face_name = image_name + f'_{idx}'
            face_prediction = recognize_face(face, known_encodings)
            #Annotating name and distance on the face image
            prediction_name, distance = face_prediction
            face_recognition = annotate_recognition(face, prediction_name, distance)
            # Saving recognition image output
            recognition_path = os.path.join(data_directory, "output_recognition")
            save_image(face_recognition, face_name + '.png', recognition_path)
    